In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

**`Load Data`**
+ `one-hot` format

In [2]:
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
print('X_train.shape :', mnist.train.images.shape)
print('Y_train.shape :', mnist.train.labels.shape)

X_train.shape : (55000, 784)
Y_train.shape : (55000, 10)


In [4]:
batch_size = 100
n_batch = mnist.train.num_examples // batch_size  # 整除

In [5]:
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

**`Dropout Param`**

In [6]:
keep_prob = tf.placeholder(tf.float32)

In [7]:
Weight_L1 = tf.Variable(tf.truncated_normal([784,2000], stddev=0.1))  # (input, output)
Biases_L1 = tf.Variable(tf.zeros([2000]) + 0.1)
Z_L1 = tf.matmul(x, Weight_L1) + Biases_L1
A_L1 = tf.nn.tanh(Z_L1)

Drop_L1 = tf.nn.dropout(A_L1, keep_prob)

In [8]:
Weight_L2 = tf.Variable(tf.truncated_normal([2000,2000], stddev=0.1))  # (input, output)
Biases_L2 = tf.Variable(tf.zeros([2000]) + 0.1)
Z_L2 = tf.matmul(Drop_L1, Weight_L2) + Biases_L2
A_L2 = tf.nn.tanh(Z_L2)

Drop_L2 = tf.nn.dropout(A_L2, keep_prob)

In [9]:
Weight_L3 = tf.Variable(tf.truncated_normal([2000,1000], stddev=0.1))  # (input, output)
Biases_L3 = tf.Variable(tf.zeros([1000]) + 0.1)
Z_L3 = tf.matmul(Drop_L2, Weight_L3) + Biases_L3
A_L3 = tf.nn.tanh(Z_L3)

Drop_L3 = tf.nn.dropout(A_L3, keep_prob)

In [10]:
Weight_L4 = tf.Variable(tf.truncated_normal([1000,10], stddev=0.1))  # (input, output)
Biases_L4 = tf.Variable(tf.zeros([10]) + 0.1)
Z_L4 = tf.matmul(Drop_L3, Weight_L4) + Biases_L4
A_L4 = tf.nn.softmax(Z_L4)

Prediction = tf.nn.softmax(Z_L4)

In [11]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=Prediction))

lr = tf.Variable(0.3)
optimizer = tf.train.GradientDescentOptimizer(lr)
train = optimizer.minimize(loss)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [12]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(Prediction, 1))
accuary = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

**由于该神经网络过于复杂，容易出现overfitting**
+ 因此使用dropout来避免过拟合
+ 当使用dropout之后
    + test accuracy的上升速度变慢，[ 从 96 -> 97 ]， 收敛速度变慢
    + **train accuracy的准确率没有很高**，**test & train accuracy 相对偏差不是很大**

In [13]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(30):
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train, feed_dict={
                x : batch_xs,
                y : batch_ys,
                keep_prob : 1.0
            })
            # keep_prob
            # 1.0 : 所有神经元都在使用
            
        test_acc = sess.run(accuary, feed_dict={
            x:mnist.test.images,
            y:mnist.test.labels,
            keep_prob : 1.0
        })
        train_acc = sess.run(accuary, feed_dict={
            x:mnist.train.images,
            y:mnist.train.labels,
            keep_prob : 1.0
        })

        print("Iter " + str(epoch) + "  Test Acuuary " + str(test_acc) + "  Train Acuuary " + str(train_acc))

Iter 0  Test Acuuary 0.8616  Train Acuuary 0.8703273
Iter 1  Test Acuuary 0.8689  Train Acuuary 0.8832
Iter 2  Test Acuuary 0.8748  Train Acuuary 0.88905454
Iter 3  Test Acuuary 0.8756  Train Acuuary 0.8920909
Iter 4  Test Acuuary 0.9651  Train Acuuary 0.98309094
Iter 5  Test Acuuary 0.9674  Train Acuuary 0.99012727
Iter 6  Test Acuuary 0.9712  Train Acuuary 0.99227273
Iter 7  Test Acuuary 0.9717  Train Acuuary 0.9931818
Iter 8  Test Acuuary 0.9727  Train Acuuary 0.99381816
Iter 9  Test Acuuary 0.9729  Train Acuuary 0.99423635
Iter 10  Test Acuuary 0.9723  Train Acuuary 0.9946727
Iter 11  Test Acuuary 0.973  Train Acuuary 0.99487275
Iter 12  Test Acuuary 0.9736  Train Acuuary 0.9951091
Iter 13  Test Acuuary 0.9739  Train Acuuary 0.9954
Iter 14  Test Acuuary 0.9741  Train Acuuary 0.9955091
Iter 15  Test Acuuary 0.974  Train Acuuary 0.9956
Iter 16  Test Acuuary 0.9743  Train Acuuary 0.9956909
Iter 17  Test Acuuary 0.9746  Train Acuuary 0.99576366
Iter 18  Test Acuuary 0.9741  Train Acuua

In [14]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(30):
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train, feed_dict={
                x : batch_xs,
                y : batch_ys,
                keep_prob : 0.7
            })
            
        test_acc = sess.run(accuary, feed_dict={
            x:mnist.test.images,
            y:mnist.test.labels,
            keep_prob : 1.0
        })
        train_acc = sess.run(accuary, feed_dict={
            x:mnist.train.images,
            y:mnist.train.labels,
            keep_prob : 1.0
        })

        print("Iter " + str(epoch) + "  Test Acuuary " + str(test_acc) + "  Train Acuuary " + str(train_acc))

Iter 0  Test Acuuary 0.9228  Train Acuuary 0.91761816
Iter 1  Test Acuuary 0.9367  Train Acuuary 0.92981815
Iter 2  Test Acuuary 0.9407  Train Acuuary 0.9398364
Iter 3  Test Acuuary 0.945  Train Acuuary 0.9450182
Iter 4  Test Acuuary 0.9473  Train Acuuary 0.94823635
Iter 5  Test Acuuary 0.951  Train Acuuary 0.95214546
Iter 6  Test Acuuary 0.9535  Train Acuuary 0.9557273
Iter 7  Test Acuuary 0.9544  Train Acuuary 0.9574
Iter 8  Test Acuuary 0.9553  Train Acuuary 0.95927274
Iter 9  Test Acuuary 0.9575  Train Acuuary 0.96154547
Iter 10  Test Acuuary 0.9604  Train Acuuary 0.96385455
Iter 11  Test Acuuary 0.961  Train Acuuary 0.96445453
Iter 12  Test Acuuary 0.9621  Train Acuuary 0.96596366
Iter 13  Test Acuuary 0.9635  Train Acuuary 0.96785456
Iter 14  Test Acuuary 0.9635  Train Acuuary 0.96894544
Iter 15  Test Acuuary 0.9637  Train Acuuary 0.96985453
Iter 16  Test Acuuary 0.9649  Train Acuuary 0.971
Iter 17  Test Acuuary 0.967  Train Acuuary 0.972
Iter 18  Test Acuuary 0.9665  Train Acuua